In [ ]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
avocado = pd.read_pickle('datasets/avoplotto.pkl')
homelessness = pd.read_csv('datasets/homelessness.csv', index_col = 0)
sales = pd.read_csv('datasets/sales_subset.csv', index_col = 0)
temperatures = pd.read_csv('datasets/temperatures.csv', index_col=0)

## Transforming DataFrames

### Inspect Dataframe

In [ ]:
print(homelessness.head())
print(homelessness.info())
print(homelessness.describe())
print(homelessness.shape)


In [ ]:
print(avocado.head())
print(avocado.info())
print(avocado.describe())
print(avocado.shape)


In [ ]:
print(sales.head())
print(sales.info())
print(sales.describe())
print(sales.shape)


In [ ]:
print(temperatures.head())
print(temperatures.info())
print(temperatures.describe())
print(temperatures.shape)

### Parts of DataFrame

In [ ]:
print(homelessness.values)
print(homelessness.columns)
print(homelessness.index)

### Sorting Rows

In [ ]:
homelessness.sort_values(by="individuals", ascending=False).head()

In [ ]:
homelessness.sort_values(by=["region","family_members"], ascending=[True, False]).head()

### Subsetting columns

In [ ]:
homelessness[["state"]].head()

In [ ]:
homelessness[["state","family_members"]].head()

### Subsetting rows

In [ ]:
homelessness[homelessness["individuals"] >= 10000].head()

In [ ]:
homelessness[homelessness["region"] == "Mountain"].head()

In [ ]:
homelessness[(homelessness["family_members"] < 1000) & (homelessness["region"] == "Pacific")].head()

### Subsetting row by categorical variables

In [ ]:
homelessness[(homelessness["region"] == "South Atlantic") | (homelessness["region"] == "Mid-Atlantic")]

**isin() method**

In [ ]:
region = ["South Atlantic","Mid=Atlantic"]

In [ ]:
homelessness[homelessness["region"].isin(region)]

### Adding new columns

In [ ]:
homelessness["total"] = homelessness["individuals"] + homelessness["family_members"]
homelessness["p_individuals"] = homelessness["individuals"] / homelessness["total"]
homelessness.head()

### Combo-attack!

In [ ]:
homelessness["indiv_per_10k"] = 10000 * homelessness["individuals"] / homelessness["state_pop"]
high_homelessness = homelessness[homelessness["indiv_per_10k"] > 20]
high_homelessness_sort = high_homelessness.sort_values(by="indiv_per_10k", ascending=False)
result = high_homelessness_sort[["state", "indiv_per_10k"]]
result.head()

## Aggregating DataFrames

In [ ]:
print(sales.head())
print(sales.info())

### Summary statistics

In [ ]:
print(sales["weekly_sales"].mean())
print(sales["weekly_sales"].median())

### Summarizing dates

In [ ]:
print(sales["date"].min())
print(sales["date"].max())

### Efficient summaries

In [ ]:
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

print(sales["temperature_c"].agg(iqr))

In [ ]:
print(sales[["temperature_c","fuel_price_usd_per_l","unemployment"]].agg([iqr,np.median]))

### Cumulative statistics

In [ ]:
sales_1_1 = sales.sort_values(by="date")
sales_1_1.head()

In [ ]:
sales_1_1["cum_weekly_sales"] = sales_1_1["weekly_sales"].cumsum()
sales_1_1["cum_max_sales"] = sales_1_1["weekly_sales"].cummax()
print(sales_1_1[["cum_weekly_sales", "cum_max_sales"]].head())

### Drop duplicates

In [ ]:
store_types = sales.drop_duplicates(subset=["store", "type"])
print(store_types.head())

In [ ]:
store_depts = sales.drop_duplicates(subset=["store","department"])
print(store_depts.head())

In [ ]:
holiday_dates = sales[sales["is_holiday"]].drop_duplicates(subset="date")
print(holiday_dates.head())

### Counting categorical variables

In [ ]:
store_counts = store_types["type"].value_counts()
print(store_counts)

store_props = store_types["type"].value_counts(normalize=True)
print(store_props)

dept_count_sort = store_depts["department"].value_counts(sort=True)
print(dept_count_sort)

dept_count_prop = store_depts["department"].value_counts(sort=True, normalize=True)
print(dept_count_prop)

### What percent of sales occurred at each store type?

In [ ]:
sales_all = sales["weekly_sales"].sum()

sales_A = sales[sales["type"] == "A"]["weekly_sales"].sum()

sales_B = sales[sales["type"] == "B"]["weekly_sales"].sum()

sales_C = sales[sales["type"] == "C"]["weekly_sales"].sum()

sales_propn_by_type = [sales_A, sales_B, sales_C] / sales_all
print(sales_propn_by_type)

In [ ]:
sales_by_type = sales.groupby("type")["weekly_sales"].sum()
sales_by_type
sales_propn_by_type = sales_by_type / sum(sales["weekly_sales"])
print(sales_propn_by_type)
sales_by_type_is_holiday = sales.groupby(["type","is_holiday"])["weekly_sales"].sum()
print(sales_by_type_is_holiday)

### Multiple grouped summaries

In [ ]:
sales_stats = sales.groupby("type")["weekly_sales"].agg([np.min,np.max,np.mean,np.median])
print(sales_stats)

unemp_fuel_stats = sales.groupby("type")["unemployment","fuel_price_usd_per_l"].agg([np.min,np.max,np.mean,np.median])
print(unemp_fuel_stats)

### Pivoting on one variable

In [ ]:
mean_sales_by_type = sales.pivot_table("weekly_sales", index="type")
print(mean_sales_by_type)

In [ ]:
mean_median_sales_by_type = sales.pivot_table(values="weekly_sales",index="type", aggfunc=[np.mean, np.median])
print(mean_median_sales_by_type)

In [ ]:
mean_sales_by_type_holiday = sales.pivot_table(values="weekly_sales", index="type", columns="is_holiday")
print(mean_sales_by_type_holiday)

### Fill in missing values and sum values with pivot tables

In [ ]:
print(sales.pivot_table(values="weekly_sales", index="department", columns="type", fill_value=0))